# Automated machine learning with Keras

In my very limited understanding of the world, the history of progress and invention has always been partially driven by laziness (read: desire to outsource), and putting a lot of effort trying to live up to the expectations created by the marketing. Take computers: here is an amazing machine that can think for us! Unfortunately, it turns out it only thinks strictly in terms of zeros and ones (for details I cannot recommend enough excellent books [But How Do It Know?](http://www.buthowdoitknow.com/) by J Clark Scott and [Code](https://en.wikipedia.org/wiki/Code:_The_Hidden_Language_of_Computer_Hardware_and_Software) by Charles Petzold), and thus a lot of people have to spend a huge amount of time and effort teaching these machines to think more like ourselves, so that we can let them do all of it it and we can start the Margarita Tuesday earlier. It is why we now have the road from zeroes and ones to the assembly language, procedural and later the object oriented languages. 

The field of machine learning/artificial intelligence seems to be following a similar path. Heralded as the tool that will revolutionize and simplify modeling and predicting, there is an awful lot of effort people need to put into it to get some level of results and value back. But, similar to programming, tools are popping up left and right that make it all easier. In this notebook we will be looking at the automated machine learning, a way to automatize the problem of hyperparameter tuning.

## Parameters of learning

The marketing way to sell machine learning is that it is a paradigm shift in modeling and/or simulating. There are three parts to any such system:

1. The model input. These describe the current state of the system, the environment or, in general, anything that we know about the system we are trying to model.
2. The internal logic - the model itself. This is a prescription that puts together whatever it received on the input and provides...
3. The output. This is a piece of data that has not been provided to the model on the input and that we want to know.

In the "traditional" paradigm someone would build the model, feed it the input data and study the output. In other words, taking the first two elements on the list the system provides us with the output to draw conclusions about. In contrast, in the case of machine learning the idea is that we provide the machine with the model inputs and the desired outputs - elements 1 and 3 - and we let it come up with the internal logic that connects the two automatically.

So far so good, but of course it's not that simple. The field of artificial intelligence - along with pretty much any other scientific field - contains a part where the people start talking about stuff being "more art than science", a part of the process that has not been (cannot be?) automatized. For artificial intelligence this is the hyperparameter tuning. Deep down, each AI model consists of a series of mathematical equations with certain parameters. The process of finding the internal logic - usually called model training - is nothing more than the computer finding the best values for these parameters. For instance, in the case of a simple linear model $y = ax + b$ the training would tell us the values for parameters $a$ and $b$ under some conditions. The issue is, already in this very simple case we had to specify that we want to use a linear equation and we also needed to come up with what it means for the model parameters $a$ and $b$ to be the best. These kinds of issues, e.g. the choice of model architecture or deciding what the model should be optimizing for, can be translated into the user choosing a series of values that are called hyperparameters. Usually, there are way fewer hyperparameters to choose than there are parameters.

Let's say we want to model out data with a simple polynomial model $y = a_0 + a_1x + a_2x^2+\dots$. The training would tell us what the parameters $a_0$ etc would be but we'd need to decide beforehand what degree of polynomial we want to use. Using the degree directly as a hyperparameter, we could write something like `quadratic_model = PolynomialModel(degree=2)` in our code to define a model $y = a_0 + a_1x + a_2x^2$ or, similarly, `deg5_model = PolynomialModel(degree=5)` for $y=\sum_{i=0}^5a_ix^i$. Thus, we have one hyperparameter - the degree of the polynomial - that we must specify, instead of having to provide many parameters of the actual model, which are found automatically in the training.

There are many different types of AI models - neural nets, random forests, support vector machines, ... - and each one comes with its own series of hyperparameters. The unfortunate thing is that there is no prescription that can tell us what combination of hyperparameters would give us the best model and, generally, multiple combinations of them need to be tried.

## Automatizing the hyperparameter search

Ideally, we would want to outsource this search for the best set of hyperparameters to the computer. There are many ways to do it. Ultimately, it's always about taking several combinations of different hyperparameter values, building a model from each, training it and looking at which performed the best. There are multiple ways of choosing these sets of hyperparameters, in various levels of smartness.

**Grid search** is perhaps the most basic. We define a set of trial values for each individual hyperparameter and then look at the model performance for all their combinations. For instance, if we want to try building my neural network with 1, 2, or 3 layers, and they all consist of 10, 20, or 30 neurons, we would run 9 iterations of the model training during the search. Grid search is useful if we have a lot of computing resources, as the number of training iterations tends to grow very quickly.

**Random search** works similarly but instead of specifying lists of values, one simply defines some distribution from which they should be randomly drawn. For instance, we might want our neural network to have a random number of layers between 1 and 10 and each layer to have a random number of neurons between 10 and 100. We specify that we want to run the search for a given number of iternations, say 20, that we deem to be enough to cover the space of hyperparameters well. The advantage here is that we can stumble upon a really good combination of hyperparametes compared to the grid search (e.g. if the best performance was at 25 neurons, we'd never find that with the grid search trying the numbers of neurons in multiples of 10), but on the other hand we are not guaranteed to find one. Furthermore, we don't really know how well we explored the hyperparameter space and, very importantly, the method doesn't learn where the more promising combinations lie and is as happy to explore the completely weird combinations of hyperparameters as the promising ones.

**Bayesian optimization** tries to address this issue. Internally it attempts to forecast the best combination of hyperaparameters to try next. It achieves this by estimating the shape of the mathematical function that ingests the hyperparameters and returns the model performance. The optimization then decides the next set of hyperparameters based on this funcition - for example, taking the one where it expects the model performance to be really good, or, alternatively, picking a combination in an area of the hyperparameter space that it does yet know very well.

A different approach is taken by the **HYPERBAND algorithm**. This is almost a random search but in addition it tries to quickly estimate whether a particular set of hyperparameers looks promising and if not, it reallocates the training resources to those hyperparameters that appear better.

There are multiple libraries that implement these algotihms and similarly multiple (machine mearning) libraries contain tools to perform the hyperparameter search using one of these.

## Hyperparameters of neural networks

Neural networks, which we will be focusing on here, come with hyperparameters left and right, e.g:

* Their architecture usually consists of layers of neurons. How many layers do I want to have?
* How many neurons do I want to have in each layer? Too few and my network does not have the "mental capacity" to learn anything, too many and the model will be too huge to train and tend to learn the data by heart instead of learning the underlying logic (also called overfitting).
* To prevent overfitting I can put "dropout" between the layers, which randomly ignores some of the neuron connections during the individual training steps. What fraction of the connections do I need to ignore - 10%? 40%? Something else?
* We need to regulate how quickly a neural network learns - too quickly and it learns the noise for each individual piece of the input data instead of the general rules; too slowly and the training would never finish. What should my learning rate be?
* ...

Generally, there is an infinite number of possibilities how to put the neurons together and train them, just as all the bricks look kind of the same but you can build an infinite number of different houses from them. In this notebook we will be looking at two libraries: `keras-tuner` and `autokeras`. These libraries, as their names suggest, use the keras wrapper around tensorflow (v2) and automatize, to various levels of the user's laziness and/or machine learning ignorance, the hyperparameter search. Specifically, we will tell the same story about finding a machine learning solution to a problem for three different "characters" with different levels of knowledge about the field. We will tackle a toy problem of classifying hand written digits in simple images (using the MNIST data set). We will progrees to higher and higher-level solutions (or lazier and lazier data scientists), starting at a still relatively tight specification, down to knowing nothing beyond the problem specification. 

**For running this notebook** you need to create a new python environment with `keras-tuner` and `autokeras` (need `pip` installation, conda doesn't seem to have it yet) libraries, as well as `matplotlib`. There is an environment file in the repo that you can use directly by running `conda env create -f automl_environment.yml`.

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical

from kerastuner.tuners import Hyperband, BayesianOptimization
from kerastuner.engine.hypermodel import HyperModel
from kerastuner.engine.hyperparameters import HyperParameters

# 1. A Data Scientist

A data scientist has a lot of knoledge about the machine learning and wants to put this to use, keeping control over how the model works so that they can efficiently deal with issues that might arise. In this section we will have a look at how `KerasTuner` can make this person's life easier, while allowing them to retain a high level of control over what is going on.

In [ ]:
# Load and prepare the MNIST data
(x, y), (val_x, val_y) = keras.datasets.mnist.load_data()
x = x.astype('float32') / 255.
val_x = val_x.astype('float32') / 255.

# our images contain one color channel, we need to explicitly create it
x = np.expand_dims(x, axis=3)
val_x = np.expand_dims(val_x, axis=3)

x = x[:10000]
y = y[:10000]


In [ ]:
# a quick look at the data
plt.imshow(x[1], cmap='Greys_r')
print(y[1])

Now we can start moving towards the automatization. First, we need an object that can generate neural networks with various sets of hyperparameters. In the presentation we were talking about a `build_model` function; here we use a class, whose `build` method corresponds to the function discussed in the talk.

In [ ]:
class MnistHyperModel(HyperModel):
    """
    Needs to contain a build method. Alternatively, just write a build function. Either needs to create a model.

    Possibly write your own run_trial method specifying how the training of the individual models goes.
    """

    def __init__(self, img_size, num_classes):
        self.img_size = img_size
        self.num_classes = num_classes

    def build(self, hp):
        """
        Build model.
        
        :param hp: hyperparameters

        NOTE: All the "hp"-s below are as of yet unspecified parameters.
            They will be chosen automatically based on our input criteria
            and the alorithm used in the tuner object during the automated
            hyperparameter search.
        """

        # define the model using the keras functional API
        input_layer = layers.Input(shape=self.img_size, name="input_layer")

        for i in range(hp.Int('num_conv_layers', min_value=2, max_value=3)):
            conv_layer = layers.Conv2D(
                filters=hp.Int(f"conv_units_{i+1}", min_value=16, max_value=64, step=16),
                kernel_size=3, # hp.Choice(f"kernel_size_{i+1}", [1, 3, 5]), # with hardcoded parameters this is of course faster
                name=f"cnn_layer_{i+1}"
            )(input_layer if i == 0 else pooling_layer) # connect the first conv layer to the input, the others to the previous pooling

            pooling_layer = layers.MaxPooling2D(name=f"pooling_layer_{i+1}")(conv_layer)

        flatten_layer = layers.Flatten(name="flatten")(pooling_layer)

        dropout_layer = layers.Dropout(
            rate=hp.Float("dropout", min_value=0., max_value=0.5, step=0.1),
            name="dropout_layer",
        )(flatten_layer)

        output_layer = layers.Dense(
            self.num_classes,
            activation='softmax',
            name="output_layer"
        )(dropout_layer)

        model = keras.models.Model(
            inputs=input_layer,
            outputs=output_layer,
        )
        
        model.compile(
            optimizer=keras.optimizers.Adam(
                hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])
            ),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'],
        )

        return model

This allows us to define a `tuner` that will take these neural networks, perform the training on our data and figure out which one of the models works best:

In [ ]:
tuner = BayesianOptimization(
    hypermodel=MnistHyperModel(img_size=(28,28,1), num_classes=10),
    objective='val_accuracy', # generally, kerastuner.Objective("my_custom_metric", direction="min")
    max_trials=10,            # number of iterations
    num_initial_points=5,     # the number of data points to estimate the shape of the hyperparameter space
    overwrite=True            # overwrite the past cached results
)

# an alternative approach
# tuner = Hyperband(
#     hypermodel=MnistHyperModel(img_size=(28,28,1), num_classes=10),
#     objective='val_accuracy',
#     max_epochs=10,
#     overwrite=True
# )

In [ ]:
# checking what we will be optimizing
tuner.search_space_summary()

In [ ]:
# perform the search
# works the same way as model.fit() would

tuner.search(
    x=x,
    y=y,
    validation_data=(val_x, val_y),
    callbacks=[
        keras.callbacks.EarlyStopping(patience=2),
        keras.callbacks.ReduceLROnPlateau(patience=5),
    ],
    verbose=1,
)


In [ ]:
# it is possible to look at the results in detail
# produces a lot of not greatly formatted output
tuner.results_summary()

In [ ]:
# best_model = tuner.get_best_models()[0] # recommended to load best hyperparams and retrain
best_hp = tuner.get_best_hyperparameters()[0]
best_model = tuner.hypermodel.build(best_hp)

In [ ]:
best_model.summary()

## 1.1 Updating search parameters

It is possible to also update the search parameters, such as if we wanted to keep a value of a particular parameter fixed. For that, we need to define an "external" hyperparameter set and feed it to the tuner explicitly.

In [ ]:
my_hps = HyperParameters()

# let's only use a single learning rate
my_hps.Fixed('learning_rate', 1e-3)

# define as before but explicitly pass hyperparameters
tuner = BayesianOptimization(
    hypermodel=MnistHyperModel(img_size=(28,28,1), num_classes=10),
    objective='val_accuracy',
    max_trials=10,
    num_initial_points=5,
    overwrite=True,
    hyperparameters=my_hps # explicity hyperparameters
)

tuner.search_space_summary()

# 2. Technical manager

We now start using `autokeras`.  This is a wrapper around `keras-tuner` which allows to abstract away a lot of thinking away from us.

A technical manager is a person that probably would not be doing quite the "data scientist" level of coding, if any. The example here is to show how autoML can help someone with a high level of understanding of machine learning concpets and how these can be translated into `autokeras` code literally concept by concept:

* the problem has to do with the computer vision
* one should not feed raw data into the model, it should be normalized in some way
* convolutional neural networks perform well in computer vision problems
* the output is classification

This is the general feature of `AutoKeras` which comes accross in the tutorials and usage - it allows you to "talk to the computer" in a very high level language. You can be very specific when you need to, handwavy elsewhere ("Try concpets X and Y.") or completely hand over the control to the alogrithms' defaults.

In [ ]:
import autokeras as ak

In [ ]:
# reload the data, with some of the previous manual steps commented out but left in to highlight the difference

(x, y), (val_x, val_y) = keras.datasets.mnist.load_data()
# x = x.astype('float32') / 255.
# val_x = val_x.astype('float32') / 255.

# x = np.expand_dims(x, axis=3)
# val_x = np.expand_dims(val_x, axis=3)

x = x[:10000]
y = y[:10000]

In [ ]:
# vaguely define model, similarly to the keras functional API
# The most frequent sentence in the autokeras documentation: "If left unspecified, it will be tuned automatically."

input_node = ak.ImageInput()
normalization = ak.Normalization()(input_node)
convolution = ak.ConvBlock()(normalization)
output_node = ak.ClassificationHead()(convolution)

auto_model = ak.AutoModel(
    inputs=input_node,
    outputs=output_node,
    overwrite=True,
    max_trials=10,
    objective='val_accuracy', # default: 'val_loss'
)

In [ ]:
# this calls keras-tuner.search

auto_model.fit(
    x=x,
    y=y,
    validation_data=(val_x, val_y),
    callbacks=[
        keras.callbacks.EarlyStopping(patience=2),
        keras.callbacks.ReduceLROnPlateau(patience=5),
    ],
    verbose=1,    
    epochs=1,
)

In [ ]:
best_model = auto_model.export_model()
best_model.summary()

# 3. First Steps Towards "True" AutoML

`AutoKeras`, for the better or the worse, allows us to go even further. While before we needed to provide at least a taste of the solution, here it allows us to only provide it with the problem definition: we have a set of input images that we want to classify into a set of classes.

In [ ]:
# using the same data as in the previous section

auto_model = ak.AutoModel(
    inputs=ak.ImageInput(),
    outputs=ak.ClassificationHead(),
    overwrite=True,
    max_trials=10, # default value
)

In [ ]:
auto_model.fit(
    x=x,
    y=y,
    validation_data=(val_x, val_y),
    callbacks=[
        keras.callbacks.EarlyStopping(patience=2),
        keras.callbacks.ReduceLROnPlateau(patience=5),
    ],
    verbose=1,    
    epochs=1,
)

As discussed in the talk, we can take this even further: AutoKeras provides ready-to-go classes for many of the most common tasks, e.g. `ImageClassifier()` or `TimeseriesForecaster()` that can be used as they are, but which also contain a lot of parameters for the user to specify.